<img src=https://brand.uark.edu/_resources/images/UA_Logo_Horizontal.jpg width="400" height="96">

###_Artificial intelligence for image processing and analysis._

# Notebook 2.3 Feature Recognition
---
##### The purpose of this notebook is showcase how we can pull out some feature within an image, and use that to identify where that feature is in an image.
##### For image processing, another useful thing that we can do with image convolution is detect _features_, or _patterns_ within an image. To determine where a specific _feature_ is in an image, we can convolve the image with that particular feature.



### Required packages
---
##### **_Run this code chunk first. If you encounter an error when trying to run code chunks in this notebook, then first try re-running this chunk._**

In [ ]:
# Import all of the necessary packages
import numpy as np
import matplotlib.pyplot as plt
import imageio as io
import torch

# Edge Detection
---
##### Sometimes, it is useful to determine in code where an object in the foreground is, compared to a background. For this type of analysis, one approach is to determine where the edge of the foreground object is, which can then be used to determine an objects location. 
##### The edges of an object can be determined with a special _[Sobel](https://en.wikipedia.org/wiki/Sobel_operator) filter_. This type of filter is used to highlight sharp changes in intensity in the image (occurring over the span of a few pixels), which can be interpreted as an edge. By doing this in two dimensions, all of the edges in an image can be highlighted.
##### In the following example, a _sobel filter_ is used to determine where the coins in the image are, relative to the background.

In [ ]:
# Load an example image
test_image = io.imread("imageio:coins.png") / 255

# Generate 2D Sobel filters
x_gradient = np.array([[-1,0,1],[-1,0,1],[-1,0,1]])
y_gradient = x_gradient.T

# Get the two intensity gradients
torch_image = torch.tensor(test_image).unsqueeze(0).unsqueeze(1).float()
torch_x_gradient = torch.tensor(x_gradient).unsqueeze(0).unsqueeze(1).float()
torch_y_gradient = torch.tensor(y_gradient).unsqueeze(0).unsqueeze(1).float()

image_x = torch.nn.functional.conv2d(torch_image,torch_x_gradient).squeeze()
image_y = torch.nn.functional.conv2d(torch_image,torch_y_gradient).squeeze()
edges = torch.sqrt((image_x**2) + (image_y**2))

# Display the...
fig = plt.figure(figsize=(15,15))

# Initial Image
ax = fig.add_subplot(2, 2, 1)
ax.set_title('Initial Image')
plot1 = plt.imshow(test_image,cmap='gray')

# X Gradient
ax = fig.add_subplot(2, 2, 2)
ax.set_title('Edges in X-Direction. Imagine a light shining from left to right.')
plot2 = plt.imshow(image_x,cmap='gray')

# Y Gradient
ax = fig.add_subplot(2, 2, 3)
ax.set_title('Edges in Y-Direction. Imagine a light shining from top to bottom.')
plot3 = plt.imshow(image_y,cmap='gray')

# All edges
ax = fig.add_subplot(2,2,4)
ax.set_title('All edges are highlighted. Notice how the outlines of all coins are highlighted.')
plot4 = plt.imshow(edges,cmap='gray')

# Text Analysis
---
##### By convolving an image with an example of the letter (or letters) that you want to pick out, you can find where that letter is in an image.
##### Here is an example image of some text:
<img src="https://raw.githubusercontent.com/imageio/imageio-binaries/master/images/page.png">

##### In the following example, a letter is picked out within the text, and used to determine where the same letter is in the entire text. Keep in mind that this is not perfect, but works relatively well.

In [ ]:
# Load an example image
test_image = io.imread("imageio:page.png") / 255

# Include form to letter to identify
# --- Form starts here --- #
#@title Choose a letter to identify within the image { run: "auto" }
letter = "e" #@param ["s","e","a"]
# --- Form ends here --- #

if letter == 's':
  kernel_size = (9,9)
  image_feature = test_image[72:72+kernel_size[0],251:251+kernel_size[1]]
  thr = 0.72
elif letter == 'e':
  kernel_size = (9,9)
  image_feature = test_image[108:108+kernel_size[0],287:287+kernel_size[1]]
  thr = 0.71
elif letter == 'a':
  kernel_size = (9,9)
  image_feature = test_image[106:106+kernel_size[0],88:88+kernel_size[1]]
  thr = 0.75

# Display the...
fig = plt.figure(figsize=(1,1))

# Letter
ax = fig.gca()
ax.set_title('Chosen letter')
plot1 = plt.imshow(image_feature,cmap='gray')

# Do some magic to the letter
image_feature = (image_feature - np.mean(image_feature)) / np.std(image_feature)
image_feature = np.conj(image_feature)

# Do some magic to the input image
test_image -= np.mean(test_image)
test_image = np.pad(test_image,(kernel_size[0]//2,kernel_size[1]//2))
corr_image = np.zeros(test_image.shape)

# Go through each pixel and see if the letter is contained there
for i in range(kernel_size[0]//2,corr_image.shape[0]-kernel_size[0]//2):
  for j in range(kernel_size[1]//2,corr_image.shape[1]-kernel_size[1]//2):
    roi = test_image[i-kernel_size[0]//2:i+kernel_size[0]//2+1,j-kernel_size[1]//2:j+kernel_size[1]//2+1]
    if np.std(roi) != 0: 
      roi = (roi - np.mean(roi)) / np.std(roi)
    else:
      roi = np.zeros(roi.shape)

    corr_image[i-kernel_size[0]//2,j-kernel_size[1]//2] = np.sum(image_feature * roi) / (kernel_size[0]*kernel_size[1])

corr_image[corr_image<0] = 0

# Find all of the places where the letter is
y,x = np.where(corr_image>thr)

# Show where the letters were found
fig = plt.figure(figsize=(13.5,13.5))
ax = plt.gca()
ax.set_title("Estimated locations of letter in the image")
im = plt.imshow(test_image,cmap='gray')
sc = plt.scatter(x+5,y+5,s=500,facecolor=[],edgecolor='r',marker='o')

# Biomedical image feature recognition
---
##### It can also be very helpful to highlight certain features within an image, which can be done be convolving the image with a small feature that is pulled out from the image.
##### Here is a common biomedical image containing a mixture of cells, including many circular red blood cells:
<img src="https://github.com/aewoessn/outreach-program-2021/blob/main/images/red_blood_cells_small.png?raw=true" alt="red_blood_cells_small.png" width="400" height="287">

##### For this particular task, we will only be using a greyscale version of this image:
<img src="https://github.com/aewoessn/outreach-program-2021/blob/main/images/red_blood_cells_small_grey.png?raw=true" alt="red_blood_cells_small.png" width="400" height="287">

##### In the following code chunk, we pull out a single blood cell from the image, and convolve that with the entire image to locate cells within the image.

In [ ]:
# Load red blood cell image from github
url = "https://github.com/aewoessn/outreach-program-2021/blob/main/images/red_blood_cells_small.png?raw=true"
test_image = np.mean(io.imread(url),axis=2) / 255

# Isolate a single red blood cell
kernel_size = (41,41)
image_feature = test_image[310:310+kernel_size[0],209:209+kernel_size[1]]

# Display the...
fig = plt.figure(figsize=(1,1))

# Image feature
ax = fig.gca()
ax.set_title('Single red blood cell feature')
plot1 = plt.imshow(image_feature,cmap='gray')

image_feature = (image_feature - np.mean(image_feature)) / np.std(image_feature)
image_feature = np.conj(image_feature)

test_image -= np.mean(test_image)
test_image = np.pad(test_image,(kernel_size[0]//2,kernel_size[1]//2))
corr_image = np.zeros(test_image.shape)

for i in range(kernel_size[0]//2,corr_image.shape[0]-kernel_size[0]//2):
  for j in range(kernel_size[1]//2,corr_image.shape[1]-kernel_size[1]//2):
    roi = test_image[i-kernel_size[0]//2:i+kernel_size[0]//2+1,j-kernel_size[1]//2:j+kernel_size[1]//2+1]
    if np.std(roi) != 0: 
      roi = (roi - np.mean(roi)) / np.std(roi)
    else:
      roi = np.zeros(roi.shape)

    corr_image[i-kernel_size[0]//2,j-kernel_size[1]//2] = np.sum(image_feature * roi) / (kernel_size[0]*kernel_size[1])

corr_image[corr_image<0] = 0
thr = 0.7

# Find all of the places where the letter is
y,x = np.where(corr_image>thr)

# Show where the letters were found
fig = plt.figure(figsize=(13.5,13.5))
ax = plt.gca()
ax.set_title("Estimated locations of red blood cells in the image")
im = plt.imshow(test_image,cmap='gray')
sc = plt.scatter(x+21,y+21,s=500,facecolor=[],edgecolor='r',marker='o')